# MS4610 Introduction to Data Analytics || Course Project 
### Data Cleaning and Augmentation
Notebook by **Group 12**

This notebook undertakes operations like correcting data types, names given to missing values, etc. Also, data columns have been (externally) given more understandable names to ease referencing (please check out the EDA notebook for information on column names). The following operations have been performed:

1. Missing value tags (missing, na, N/A) replaced with `numpy.nan`
2. Label encoding some categorical columns and typecasting to appropriate dtypes 

**NOTE**: Synthetic imputation of missing data has been performed externally through normal Python scripts. Since these processes are computationally very expensive and time consuming, they were run on **Google Colab** kernels with GPU support. The code files are available in the main repository.

In [2]:
# Data handling and transformation libraries

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Imputation libraries

from impyute.imputation.cs import mice
from impyute.imputation.cs import fast_knn
from missingpy import MissForest

# Resampling library

# from imblearn.over_sampling import SMOTE as smote

# Other libraries

import sys
import warnings

sys.setrecursionlimit(1000000)
warnings.filterwarnings("ignore")

print("Dependencies loaded")

Dependencies loaded


In [2]:
# Load training dataset

train = pd.read_csv('.././data/train.csv')
train_mf = pd.read_csv("/home/nishant/Desktop/IDA Project/mod_data/train_mf.csv")
train_mf_res = pd.read_csv("/home/nishant/Desktop/IDA Project/mod_data/train_mf_res.csv")

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83000 entries, 0 to 82999
Data columns (total 50 columns):
application_key          83000 non-null int64
credit_score             83000 non-null object
risk_score               77114 non-null float64
sev_def_any              82465 non-null float64
sev_def_auto             82465 non-null float64
sev_def_edu              82465 non-null float64
min_credit_rev           83000 non-null object
max_credit_act           83000 non-null object
max_credit_act_rev       83000 non-null object
total_credit_1_miss      83000 non-null object
total_credit             83000 non-null object
due_collected            83000 non-null object
total_due                83000 non-null object
annual_pay               83000 non-null object
annual_income            83000 non-null int64
property_value           83000 non-null object
fc_cards_act_rev         83000 non-null object
fc_cards_act             83000 non-null object
fc_lines_act             83000 non-null obj

## Useful Functions
In this section, we have defined some functions that are commonly used in data cleaning. This makes readability of code and repeatability of operations convenient down the line. Available functions:
1. **missing_table**: Tally of missing values in the dataset by column, arranged in descending order.
2. **drop_bad_rows**: Drops rows with more percentage of null values than specified threshold
3. **drop_bad_cols**: Drops columns with more percentage of null values than specified threshold
4. **create_embedding**: Generates embeddings for categorical features when provided suitable arguments. See Model Selection notebook for model details about categorical feature embeddings.
5. **scale_df**: Standard scaling function for passed columns of a DataFrame 

In [3]:
def missing_table(df, threshold=None, ascending=False):
    """
    Counts number of missing values and percentage of missing values in every column
    of input pandas DataFrame object.
    
    :params: 
        threshold: (int/float) returns only those columns with number/percentage of missing
                   values higher than this value, default is None (returns all columns)
        ascending: (boolean) sorts table in ascending order of missing values
                   if set to True, default is False
                   
    :return: columns with missing values above threshold; pandas DataFrame object with column 
             name, number of missing values and percentage of missing values
    """
    cols = df.columns
    miss_vals = np.array([df[col].isnull().sum() for col in cols])
    miss_vals_percent = (miss_vals / len(df))*100
    
    miss_table = pd.DataFrame(np.vstack((cols, miss_vals, miss_vals_percent)).T,
                              columns=['column', 'missing values', '% missing values'])
    
    if threshold is None:
        miss_table = miss_table.sort_values(by='missing values', ascending=ascending)
        return miss_table
    else:
        if threshold <= 1.0:
            ret = miss_table.loc[miss_table['% missing values'] >= threshold*100.0, :]
            return ret.sort_values(by='missing values', ascending=ascending)
        elif threshold > 1.0:
            ret = miss_table.loc[miss_table['missing values'] >= threshold, :]
            return ret.sort_values(by='missing values', ascending=ascending)
        else:
            raise ValueError('Invalid threshold type')
            
            
def drop_bad_rows(df, target=None, threshold=0.8):
    
    bad_rows = train.loc[train.isnull().sum(axis=1)/train.shape[1]>=threshold, :]
    if target is not None:
        class0, class1 = bad_rows[target].value_counts().values[0], bad_rows[target].value_counts().values[1]
        class0_pc, class1_pc = class0/len(df), class1/len(df)
        print("Dropped %d (%.4f%%) class 0 and %d (%.4f%%) class 1 examples" % 
              (class0, class0_pc*100, class1, class1_pc*100))
    df_new = df.drop(bad_rows.index, axis=0)
    return df_new


def drop_bad_cols(df, threshold=0.4):
    
    miss_table = missing_table(df, threshold=threshold)
    bad_cols = miss_table['column'].values.tolist()
    df_new = df.drop(bad_cols, axis=1)
    print("Dropped %d of %d columns" % (len(bad_cols), df.shape[1]))
    return df_new


def create_embedding(train_mf, test_mf, col, batch_size=100):
    
    # Dependencies

    import tensorflow as tf
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten
    from keras.utils import to_categorical
    from keras.layers.embeddings import Embedding
    
    # EMBEDDING ==========================================================================================

    labels = train_mf.default_ind.values

    le = LabelEncoder()
    train_mf[col] = le.fit_transform(train_mf[col])
    test_mf[col] = le.transform(test_mf[col])

    vocab_size = train_mf[col].nunique()
    max_length = 1
    embed_vec_size = min(50, (train_mf[col].nunique()+1)//2)
    col_train = train_mf[col].values
    col_test = test_mf[col].values

    # Define model
    print('[INFO] Generating embeddings')
    model = Sequential()
    model.add(Embedding(vocab_size, embed_vec_size, input_length=max_length))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    model.fit(col_train, labels, epochs=1, batch_size=batch_size)

    # Get outputs of first layer
    new_model = Sequential()
    new_model.add(Embedding(vocab_size, embed_vec_size, input_length=max_length))

    new_model.set_weights(model.layers[0].get_weights())
    new_model.compile(optimizer='adam', loss='binary_crossentropy')

    preds_train = new_model.predict(col_train).reshape(-1, embed_vec_size)
    preds_test = new_model.predict(col_test).reshape(-1, embed_vec_size)
    embed_cols_train = [str(col)+'_emb'+str(i) for i in range(preds_train.shape[1])]
    embed_cols_test = [str(col)+'_emb'+str(i) for i in range(preds_test.shape[1])]
    col_embed_df_train = pd.DataFrame(preds_train, columns=embed_cols_train)
    col_embed_df_test = pd.DataFrame(preds_test, columns=embed_cols_test)

    return col_embed_df_train, col_embed_df_test


def scale_df(train_mf, test_mf, non_scale_cols, replace_cols):

    from sklearn.preprocessing import StandardScaler
    
    ss = StandardScaler()
    X_sc_train = ss.fit_transform(train_mf.drop(non_scale_cols, axis=1).values)
    X_sc_test = ss.transform(test_mf.drop(non_scale_cols, axis=1).values)

    X_df_train = pd.DataFrame(X_sc_train)
    X_df_train[replace_cols] = train_mf[replace_cols].values

    X_df_test = pd.DataFrame(X_sc_test)
    X_df_test[replace_cols] = test_mf[replace_cols].values
    
    return X_df_train, X_df_test

## Missing values tag correction
Missing values have been represented with variety of strings. It is difficult to deal with them during data exploration with `pandas`. We will convert all of them into numpy not-a-number values.

In [ ]:
# NOTE: 
# Looking for a vectorized implementation of this operation
# The code below takes about 10 seconds to run, which is quite slow

miss_tags = ['missing', 'na', 'N/A']

for col in train.columns:
    for i in range(len(train)):
        if train.at[i, col] in miss_tags:
            train.at[i, col] = np.nan

# train = train.replace(miss_tags, [np.nan, np.nan, np.nan])

This operation now gives a clearer picture of the dataset when accessed using `info()` attribute of the DataFrame. Very few columns have all non-null values.

In [5]:
# DataFrame information post null value tagging

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83000 entries, 0 to 82999
Data columns (total 50 columns):
application_key          83000 non-null int64
credit_score             79267 non-null object
risk_score               77114 non-null float64
sev_def_any              82465 non-null float64
sev_def_auto             82465 non-null float64
sev_def_edu              82465 non-null float64
min_credit_rev           63299 non-null object
max_credit_act           75326 non-null object
max_credit_act_rev       63291 non-null object
total_credit_1_miss      71318 non-null object
total_credit             82465 non-null object
due_collected            36283 non-null object
total_due                68422 non-null object
annual_pay               73311 non-null object
annual_income            83000 non-null int64
property_value           49481 non-null object
fc_cards_act_rev         63757 non-null object
fc_cards_act             66501 non-null object
fc_lines_act             67641 non-null obj

## Label Encoding and Typecasting
Some columns have type `object` and need encoding into `float` types so that further processing on them be possible. In this section, we perform relevant transformations.

In [6]:
# Label encoding categorical columns

enc_columns = ['card_type']

le = LabelEncoder()
train[enc_columns] = le.fit_transform(train[enc_columns])

In [7]:
# Converting all columns to float data type
# We only have two categorical columns (card_type and location_id) which we can 
# change back to categorical type later

for col in train.columns:
    train[col] = train[col].astype('float')

In [8]:
# First, we will remove some rows and columns with too many missing values

train = drop_bad_rows(train, target='default_ind', threshold=0.75)
train = drop_bad_cols(train, threshold=0.4)

Dropped 378 (0.4554%) class 0 and 157 (0.1892%) class 1 examples
Dropped 9 of 50 columns


## Missing value imputation
In this section, we will try some methods for synthetic imputation of missing values in the dataset. Based on model performance on these imputed datasets, we will choose the best imputation method. The following methods have been used.
1. **Zero imputation**: All missing values replaced with zero
2. **Missing Forest imputation**: A random forest imputes each column assuming it's the target and the other columns are features

In [10]:
# Zero imputation

train_zero = train.fillna(0)
train_zero.to_csv(".././mod_data/train_zero.csv", index=False)
train.to_csv(".././mod_data/train.csv", index=False)

In [ ]:
# Missing Forest imputation
# NOTE (DO NOT RUN ON JUPYTER):
# This imputation is very computationally expensive and time consuming
# Was performed externally on PyCharm

cols = train.columns.tolist()

# Impute values
# Function returns a numpy ndarray, which we convert to DataFrame again
imputer = MissForest()

print("[INFO] Imputation started")
X_imputed = imputer.fit_transform(train.values)

print("[INFO] Imputation complete")
train_mf = pd.DataFrame(X_imputed, columns=cols)

# Save new DataFrame to drive
train_mf.to_csv("/home/nishant/Desktop/IDA Project/mod_data/train_mf.csv", index=False)

## Resampling minority class using SMOTE
**Synthetic Minority Over-sampling Technique (SMOTE)** is a novel method for reducing/removing class imbalance in datasets. In the cell below, we perform SMOTE resampling on Missing Forest imputed data to obtained a resampled dataset with equal representation of defaulters and non-defaulters.

In [6]:
# Import data

cols = train_mf.columns.tolist()
y = train_mf.default_ind.values
X = train_mf.drop(['default_ind'], axis=1).values

# SMOTE resampling
sm = smote(sampling_strategy='auto', random_state=123)
X_res, y_res = sm.fit_resample(X, y)

pc_increase = (X_res.shape[0] - X.shape[0])*100/X.shape[0]
print("Increase in dataset size: %.2f%% (%d rows added)" % (pc_increase, X_res.shape[0] - X.shape[0]))

# Reform dataframe
res_df_vals = np.hstack((X_res, y_res.reshape((-1, 1))))
res_df = pd.DataFrame(res_df_vals, columns=cols)

res_df = res_df.sample(frac=1).reset_index(drop=True)

res_df.to_csv("/home/nishant/Desktop/IDA Project/mod_data/train_mf_res.csv", index=False)

Increase in dataset size: 42.53% (35069 rows added)


## Binning continuous variables
In this final attempt, we discretize all continuous variables by making quantile cuts on each continuous column. This will help us obtain as many categorical variables. Using these, we will test the performance of CatBoost and enitity embeddings with neural networks. **Motivation**: Our target is categorical and we are relying mostly on decision tree models. These models find decision rules to make predictions by computing thresholds for minimising entropy after split for continuous data. By reducing the number of unique values, we make the splitting procedure easier and more semantically interpretable. Let's see if it improves model performance also. 

In [4]:
train_mf_res.head()

,application_key,credit_score,risk_score,sev_def_any,sev_def_auto,sev_def_edu,min_credit_rev,max_credit_act,max_credit_act_rev,total_credit_1_miss,...,lines_deli,stress_index,lines_high_risk,max_due_ratio,auto_2_miss,card_type,location_id,default_ind,magic_uid,loc_woe
0,236289.000000,1681.000000,1.281100,3.076,1.695,0.0,10.00000,5064.000000,2007.000000,2775.000000,...,0.0,0.54545,1.000000,0.948490,0.0,1.0,3247.0,0.0,61,-0.633713
1,354149.000000,1889.000000,0.503900,0.000,0.000,0.0,527.00000,17722.000000,17722.000000,21881.000000,...,0.0,0.00000,16.000000,0.424760,0.0,0.0,111.0,0.0,69,-0.992980
2,272397.497773,1718.442313,1.729757,0.000,0.000,0.0,20.33653,2423.463499,2423.463499,23706.328874,...,0.0,0.00000,20.810437,0.155529,0.0,0.0,712.0,1.0,71,0.908190
3,244594.000000,1599.000000,0.000000,13.790,0.000,0.0,0.00000,0.000000,0.000000,1444.000000,...,0.0,0.95652,0.000000,1.000000,0.0,0.0,8.0,0.0,60,-1.016777
4,269409.000000,1883.000000,0.439200,0.000,0.000,0.0,7730.00000,19820.000000,7730.000000,44605.000000,...,0.0,0.00000,3.000000,1.000000,0.0,0.0,22.0,0.0,2,-0.971495


In [3]:
# Isolate numeric columns and perform qcuts

num_cols = train_mf_res.drop(['application_key', 'card_type', 'location_id', 'default_ind'], axis=1).columns.tolist()
train_bin = train_mf_res.drop(num_cols, axis=1)

for col in num_cols:
    enc_col = pd.cut(train_mf_res[col], 10, labels=[str(i+1) for i in range(10)], duplicates='drop')
    train_bin[str(col)] = enc_col 

In [4]:
# Type conversion for some columns

train_bin['card_type'] = train_bin.card_type.apply(np.round).astype('int')
train_bin['location_id'] = train_bin.location_id.astype('int')
train_bin['default_ind'] = train_bin.default_ind.astype('int')

In [5]:
# Convert categorical columns to appropriate type

cat_cols = train_bin.drop('application_key', axis=1).columns.tolist() 
train_bin[cat_cols] = train_bin[cat_cols].astype('category')

In [6]:
train_bin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117534 entries, 0 to 117533
Data columns (total 43 columns):
application_key          117534 non-null float64
card_type                117534 non-null category
location_id              117534 non-null category
default_ind              117534 non-null category
credit_score             117534 non-null category
risk_score               117534 non-null category
sev_def_any              117534 non-null category
sev_def_auto             117534 non-null category
sev_def_edu              117534 non-null category
min_credit_rev           117534 non-null category
max_credit_act           117534 non-null category
max_credit_act_rev       117534 non-null category
total_credit_1_miss      117534 non-null category
total_credit             117534 non-null category
total_due                117534 non-null category
annual_pay               117534 non-null category
annual_income            117534 non-null category
fc_cards_act_rev         117534 non-null

In [7]:
# Save this df to drive

train_bin.to_csv(".././mod_data/train_bin.csv", index=False)

## SMOTE after transforming categorical columns
Since SMOTE converts our categorical columns to continuous if provided in their raw form, we will first convert our categorical columns to continuous by performing entity embeddings and then apply SMOTE. This way our columns will always remain continuous.

In [80]:
# Load missing forest imputed data and split into train and test

train_mf = pd.read_csv(".././mod_data/train_mf.csv")
col_names = train_mf.columns.tolist()

train_final = train_mf.values[:int(0.8*len(train_mf))]
test_final = train_mf.values[int(0.8*len(train_mf)):]

train_final = pd.DataFrame(train_final, columns=col_names).drop('application_key', axis=1)
test_final = pd.DataFrame(test_final, columns=col_names).drop('application_key', axis=1)

In [84]:
# Get embeddings

loc_embed_df_train, loc_embed_df_test = create_embedding(train_final, test_final, 'location_id')
card_embed_df_train, card_embed_df_test = create_embedding(train_final, test_final, 'card_type')

# Scale numerical columns

non_scale_cols = ['card_type', 'location_id', 'default_ind']
X_df_train, X_df_test = scale_df(train_final, test_final, non_scale_cols, replace_cols='card_type')

[INFO] Generating embeddings
Epoch 1/1
65972/65972 [==============================] - 1s 16us/step - loss: 0.6057 - binary_accuracy: 0.7112
[INFO] Generating embeddings
Epoch 1/1
65972/65972 [==============================] - 1s 15us/step - loss: 0.6240 - binary_accuracy: 0.7144


In [85]:
# Replace categorical columns by embeddings 

train_def_vals = train_final.default_ind.values
train_final = pd.concat([X_df_train, loc_embed_df_train, card_embed_df_train], axis=1)
train_final['default_ind'] = train_def_vals

test_def_vals = test_final.default_ind.values
test_final = pd.concat([X_df_test, loc_embed_df_test, card_embed_df_test], axis=1)
test_final['default_ind'] = test_def_vals

In [86]:
# Perform SMOTE on training dataset

cols = train_final.columns.tolist()
y = train_final.default_ind.values
X = train_final.drop(['default_ind'], axis=1).values

# SMOTE resampling
sm = smote(sampling_strategy='minority', random_state=123)
X_res, y_res = sm.fit_resample(X, y)

pc_increase = (X_res.shape[0] - X.shape[0])*100/X.shape[0]
print("Increase in dataset size: %.2f%% (%d rows added)" % (pc_increase, X_res.shape[0] - X.shape[0]))

# Reform dataframe
res_df_vals = np.hstack((X_res, y_res.reshape((-1, 1))))
res_df = pd.DataFrame(res_df_vals, columns=cols)

train_final = res_df.sample(frac=1).reset_index(drop=True)

Increase in dataset size: 42.88% (28290 rows added)


In [87]:
# Save files to disk

train_final.to_csv(".././mod_data/train_final.csv", index=False)
test_final.to_csv(".././mod_data/test_final.csv", index=False)

## Random over and under sampling
Since SMOTE is causing issues related to data types and overall gives no benefit on test accuracy, we will try random over-sampling/under-sampling to see if that helps. Here, data types will remain consistent, so we know that our model is ready for the test data in it's raw form.

In [3]:
# Import missForest imputed dataset

train_mf = pd.read_csv(".././mod_data/train_mf.csv")

In [4]:
# Split train_mf into permanent train and test

col_names = train_mf.columns.tolist()

train_rs = pd.DataFrame(train_mf.values[:int(0.8*len(train_mf))], columns=col_names)
test_rs = pd.DataFrame(train_mf.values[int(0.8*len(train_mf)):], columns=col_names)

In [9]:
# Random oversample and undersample in train_rs

count_class_0, count_class_1 = train_rs.default_ind.value_counts()
train_rs_class_0 = train_rs[train_rs.default_ind == 0]
train_rs_class_1 = train_rs[train_rs.default_ind == 1]

class_1_over = train_rs_class_1.sample(count_class_0, replace=True)
train_rs_over = pd.concat([train_rs_class_0, class_1_over], axis=0)

class_0_under = train_rs_class_0.sample(count_class_1, replace=True)
train_rs_under = pd.concat([class_0_under, train_rs_class_1], axis=0)

In [12]:
# Drop application key columns and save dataFrames 

train_rs_over = train_rs_over.drop('application_key', axis=1)
train_rs_under = train_rs_under.drop('application_key', axis=1)
test_rs = test_rs.drop('application_key', axis=1)

train_rs_over.to_csv("/home/nishant/Desktop/IDA Project/rs_data/train_os.csv", index=False)
train_rs_under.to_csv("/home/nishant/Desktop/IDA Project/rs_data/train_us.csv", index=False)
test_rs.to_csv("/home/nishant/Desktop/IDA Project/rs_data/test_rs.csv", index=False)

In [16]:
train_rs_over.to_csv("/home/nishant/Desktop/IDA Project/rs_data/train_os.tsv", sep='\t', index=False)
train_rs_under.to_csv("/home/nishant/Desktop/IDA Project/rs_data/train_us.tsv", sep='\t', index=False)
test_rs.to_csv("/home/nishant/Desktop/IDA Project/rs_data/test_rs.tsv", sep='\t', index=False)

## Leaderboard dataset cleaning
Final operations for the leaderboard dataset have been performed here. Operations have been decided based on model performance on differently modified datasets in the Model Selection notebook.

In [3]:
# Test dataset final

test_final = pd.read_csv(".././data/testX.csv")

In [4]:
test_final.head()

,application_key,credit_score,risk_score,sev_def_any,sev_def_auto,sev_def_edu,min_credit_rev,max_credit_act,max_credit_act_rev,total_credit_1_miss,...,lines_deli,line_util_edu,line_util_auto,stress_index,lines_high_risk,max_due_ratio,mort_2_miss,auto_2_miss,card_type,location_id
0,578069,1719,0.6174,8.623,0.0,0.0,258,258,258,10729,...,0,missing,missing,0.54545,2,0.91837,0,0,C,3247
1,578070,1795,0.2051,0.000,0.0,0.0,1685,12711,8913,80519,...,0,missing,missing,0.17241,4,0.94563,0,0,C,18
2,578071,1742,0.5082,0.000,0.0,0.0,1185,8954,8954,1189,...,0,missing,missing,0.64706,1,0.97054,0,0,C,11
3,578072,1685,0.2595,25.409,0.0,0.0,missing,3354,missing,missing,...,0,missing,missing,0.85714,1,1.00000,na,0,C,32
4,578073,1666,1.2678,0.000,0.0,0.0,570,570,570,missing,...,0,missing,101.61,missing,0,0.99617,na,0,L,89


In [6]:
miss_tags = ['missing', 'na', 'N/A']

for col in test_final.columns:
    for i in range(len(test_final)):
        if test_final.at[i, col] in miss_tags:
            test_final.at[i, col] = np.nan

In [7]:
test_final.head()

,application_key,credit_score,risk_score,sev_def_any,sev_def_auto,sev_def_edu,min_credit_rev,max_credit_act,max_credit_act_rev,total_credit_1_miss,...,lines_deli,line_util_edu,line_util_auto,stress_index,lines_high_risk,max_due_ratio,mort_2_miss,auto_2_miss,card_type,location_id
0,578069,1719,0.6174,8.623,0.0,0.0,258,258,258,10729,...,0,NaN,NaN,0.54545,2,0.91837,0,0,C,3247
1,578070,1795,0.2051,0.000,0.0,0.0,1685,12711,8913,80519,...,0,NaN,NaN,0.17241,4,0.94563,0,0,C,18
2,578071,1742,0.5082,0.000,0.0,0.0,1185,8954,8954,1189,...,0,NaN,NaN,0.64706,1,0.97054,0,0,C,11
3,578072,1685,0.2595,25.409,0.0,0.0,NaN,3354,NaN,NaN,...,0,NaN,NaN,0.85714,1,1.00000,NaN,0,C,32
4,578073,1666,1.2678,0.000,0.0,0.0,570,570,570,NaN,...,0,NaN,101.61,NaN,0,0.99617,NaN,0,L,89


In [20]:
test_final.to_csv(".././mod_data/test_final.csv", index=False)

In [15]:
# Label encode card type
card_map = {'C': 0.0, 'L': 1.0}
test_final['card_type'] = test_final['card_type'].map(card_map)

In [18]:
cols = test_final.columns
test_final[cols]=test_final[cols].astype('float')

In [11]:
train_mod = pd.read_csv(".././mod_data/train.csv")

In [12]:
train = pd.read_csv(".././data/train.csv")